# Definción de la clase platform

In [1]:
import numpy as np
import cmath
import pyperclip
import geogebra
import limb
import movement_assignment
import checks
import basis
import design_utils

In [2]:
class platform:
    def __init__(self,d,D,l1,l2,max_limb_error,max_limb_try_count,max_joint_angle):
        "d: distancia de cada junta universal en la plataforma al centro"
        
        "D: distancia del eje de cada motor al centro de la base"
        
        "la altura del eje del motor es conciderada el plano z=0"
        
        "l1: long del antebrazo"
        
        "l2: long del brazo"
        
        "max_limb_error: máximo error (en las mismas unidades de l1 y l2) que es permitido para posicionar el brazo"
        
        "max_limb_try_count: máxima cantidad de iteraciones para calcular el ángulo de un motor, si la cantidad"
        "Si la cantidad requerida de iteraciones es mayor, genera un error llamado 'Point unreachable' "
        
        "max_joint_angle:angulo máximo que puede tener la junta universal, una posición que lo exceda genera un error"
        "llamado 'Joint angle exceeded'"
        
        self.c1=np.array([0,-d,0])
        self.c2=np.array([-d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        self.c3=np.array([d*np.sin(np.pi/3),d*np.cos(np.pi/3),0])
        
        self.d=d
        self.D=D

        self.l1=l1
        self.l2=l2
        
        self.max_limb_error=max_limb_error
        self.max_limb_try_count=max_limb_try_count
        
        #defino los angulos de los motores
        self.ang1=0
        self.ang2=0
        self.ang3=0
        #defino los angulos máximos que se les permite rotar a los motores
        self.angle_max=90
        self.angle_min=-90
        #defino el ángulo máximo permitido en las juntas universales
        self.max_joint_angle=max_joint_angle        

    def reset_platform (self):
        #función que vuelve a la plataforma a su estado sin actuar
        self.c1=self.d*np.array([0,-1,0])
        self.c2=self.d*np.array([-1*np.sin(np.pi/3),np.cos(np.pi/3),0])
        self.c3=self.d*np.array([np.sin(np.pi/3),np.cos(np.pi/3),0])
        #defino los angulos de los motores
        self.ang1=0
        self.ang2=0
        self.ang3=0
        
    def rotate(self,movement,joint,angle):
        #Cambio la base de referencia a la de la junta en la cual se realiza el movimiento
        if joint=="C1":
            self.c1=basis.base_change_cannon_to_m1(self.c1)
            self.c2=basis.base_change_cannon_to_m1(self.c2)
            self.c3=basis.base_change_cannon_to_m1(self.c3)              
        if joint=="C2":
            self.c1=basis.base_change_cannon_to_m2(self.c1)
            self.c2=basis.base_change_cannon_to_m2(self.c2)
            self.c3=basis.base_change_cannon_to_m2(self.c3)                 
        if joint=="C3":
            self.c1=basis.base_change_cannon_to_m3(self.c1)
            self.c2=basis.base_change_cannon_to_m3(self.c2)
            self.c3=basis.base_change_cannon_to_m3(self.c3)
        #Realizo la rotación
        if movement=="pitch":
            self.c1=basis.rotate_in_x_axis(self.c1,angle)
            self.c2=basis.rotate_in_x_axis(self.c2,angle)
            self.c3=basis.rotate_in_x_axis(self.c3,angle)    
        if movement=="roll":
            self.c1=basis.rotate_in_y_axis(self.c1,angle)
            self.c2=basis.rotate_in_y_axis(self.c2,angle)
            self.c3=basis.rotate_in_y_axis(self.c3,angle)
            
        #Corrijo el offset de la plataforma
        ##Siempre tomo la junta de la izquiera como referencia en cada caso
        if joint=="C1":
            self.apply_offset(self.c2)
        if joint=="C2":
            self.apply_offset(self.c3)
        if joint=="C3":
            self.apply_offset(self.c1)
        #Cambiamos a la base orginal
        if joint=="C1":
            self.c1=basis.base_change_m1_to_cannon(self.c1)
            self.c2=basis.base_change_m1_to_cannon(self.c2)
            self.c3=basis.base_change_m1_to_cannon(self.c3)              
        if joint=="C2":
            self.c1=basis.base_change_m2_to_cannon(self.c1)
            self.c2=basis.base_change_m2_to_cannon(self.c2)
            self.c3=basis.base_change_m2_to_cannon(self.c3)                 
        if joint=="C3":
            self.c1=basis.base_change_m3_to_cannon(self.c1)
            self.c2=basis.base_change_m3_to_cannon(self.c2)
            self.c3=basis.base_change_m3_to_cannon(self.c3)           
        
    def apply_offset (self,pos):
        "la función desplaza la plataforma el offset correspondiente para compenzar el introducido por la rotación"
        allowed_pos=-pos[0]*np.tan(np.pi/6)
        offset=np.array([0,allowed_pos-pos[1],0])
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        
        
    def calculate_P(self):
        #Función que obtiene las coordenadas de las juntas c1,c2,c3 en las bases de los motores m1,m2 y m3
        #respectivamente. Se le suma el vector (0,D,0) para establecer el punto de origen en el rotor del motor
        P1=basis.base_change_cannon_to_m1(self.c1)+np.array([0,self.D,0])
        P2=basis.base_change_cannon_to_m2(self.c2)+np.array([0,self.D,0])
        P3=basis.base_change_cannon_to_m3(self.c3)+np.array([0,self.D,0])
        #elimino la coordenana en x. Dado que las juntas se desplazan por el plano y-z de cada motor la coordenada en x es siempre cero
        P1=np.array([P1[1],P1[2]])
        P2=np.array([P2[1],P2[2]])
        P3=np.array([P3[1],P3[2]])
        return P1,P2,P3
    

        
    def correct_height(self,h):
        #Función que dada la posición de las 3 juntas en el espacio cálcula la altura del centro de la plataforma
        #y desplaza las juntas verticalmente para cumplir con la altura solicitada
        
        #Obtengo las componentes x,y,z de las juntas
        c1x=self.c1[0]
        c2x=self.c2[0]
        c3x=self.c3[0]
        
        c1y=self.c1[1]
        c2y=self.c2[1]
        c3y=self.c3[1]
        
        c1z=self.c1[2]
        c2z=self.c2[2]
        c3z=self.c3[2]
        #Dada la ecuación del plano Z= q1 * x + q2 * y + q3, donde q1,2,3 son los parametros que definen el plano
        #Es posible encontrar el valor de los 3 parametros si se poseen 3 puntos que definan el plano 
        #Particularmente el parametro q3 nos da la altura del plano en el centro de la plataforma q3=Z(x=0,y=0)
        #A continuación lo que se hace es resolver el sistema matricial con los 3 puntos siendo las juntas 
        #y los parametros del plano siendo la variable params. El parametro q3 es params[2]
        A=np.array([[c1x,c1y,1],[c2x,c2y,1],[c3x,c3y,1]])
        B=np.array([c1z,c2z,c3z]).transpose()
        params=np.linalg.solve(A,B)
        #Obtengo un vector de offset que sumado a las juntas las desplace a la posición deseada
        offset=np.array([0,0,h-params[2]])
        #corrijo la altura
        self.c1=self.c1+offset
        self.c2=self.c2+offset
        self.c3=self.c3+offset
        

    def solve_platform(self,r_x,r_y,h):
        #Función que dados los ángulos en x e y y la altura deseada, obtiene los ángulos de los motores para
        #alcanzarlo
        
        self.reset_platform() #pongo las juntas en la posición inicial para empezar el analisis
        if r_x==0 and r_y==0:
            #Este if se utiliza para no calcular rotaciones cuando los ángulos de ambos ejes son cero
            self.correct_height(h)
        else:
            joint,movement,angle=movement_assignment.assign_movement(r_x,r_y)
            #assign_movement calcula la junta, el movimiento y el ángulo necesario para realizar de forma
            #aproximada los angulos requeridos en x y en y
            self.rotate(movement,joint,angle)
            self.correct_height(h) #llevo la plataforma a la altura indicada
        
        P1,P2,P3=self.calculate_P() #referencio las 3 juntas C1,C2,C3 a los planos de referencia de su respectivo
                                    #motor, tomando el rotor del motor como punto (0,0). El analisis se realiza en 
                                    #dos dimensiones dado que las juntas se mueven sobre un plano. Las juntas referenciadas
                                    #pasan a llamarse P1,P2 y P3 respectivamente
                    
        limb_i=limb.limb(self.l1,self.l2) #defino el objeto que representa cada brazo de la plataforma
        limb_2=limb.limb(self.l1,12.5) #defino el objeto que representa cada brazo de la plataforma
        
        #Calculo los angulos necesarios en cada motor dado un error máximo en cm de la posición del punto P1,2,3
        #Y un número máximo de iteraciones posibles para llegar al resultado
        self.ang1=limb_i.calculate_motor_angle(P1,self.max_limb_error,self.max_limb_try_count)
        self.ang2=limb_2.calculate_motor_angle(P2,self.max_limb_error,self.max_limb_try_count)
        self.ang3=limb_i.calculate_motor_angle(P3,self.max_limb_error,self.max_limb_try_count)
         #Chekeo que el ángulo en las juntas universales no supere el máximo que permiten
        for joint in ["C1","C2","C3"]:
            angle=checks.get_joint_angle(self.c1,self.c2,self.c3,joint)
            if np.abs(angle)> self.max_joint_angle:
                #raise ValueError('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                print('Joint angle exceeded\n {}.angle={}º >{}º'.format(joint,angle*180/np.pi,self.max_joint_angle*180/np.pi))
                return -1
            else:
                return 0       

# Testeo de la plataforma

## Checkeo de movimientos posibles y generación de nube de puntos

In [9]:
#definición de rangos de recorrido
angle_spacex=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
angle_spacey=np.linspace(start=-19*np.pi/180,stop=19*np.pi/180,num=10)
h_space=[8]
#Creación de objeto plataforma
p1=platform(d=6,D=10,l1=6,l2=12,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
#simulación de los movimientos posibles
s1=design_utils.test_movements(platform=p1,h_range=h_space,r_x_range=angle_spacex,r_y_range=angle_spacey)

angle=45.0
angle=37.456473007566316
angle=28.42122951305713
angle=17.866112351683487
angle=6.110614056804322
roll_vector: [1. 0. 0.]
angle=-6.110614056804332
roll_vector: [ 1.0000000e+00 -1.2246468e-16 -0.0000000e+00]
angle=-17.866112351683494
angle=-28.42122951305713
angle=-37.456473007566316
angle=-45.0
Motor choca con el plato:

angle=52.543526992433684
angle=45.0
angle=35.23684796881902
angle=22.818472108019943
angle=7.9549137416280535
roll_vector: [1. 0. 0.]
angle=-7.954913741628067
roll_vector: [ 1.0000000e+00 -1.2246468e-16 -0.0000000e+00]
angle=-22.818472108019954
angle=-35.23684796881902
angle=-45.00000000000001
angle=-52.543526992433684
Motor choca con el plato:

angle=61.57877048694288
angle=54.76315203118098
angle=45.0
angle=30.77905891453861
angle=11.189875272761293
roll_vector: [1. 0. 0.]
angle=-11.189875272761311
roll_vector: [ 1.0000000e+00 -1.2246468e-16 -0.0000000e+00]
angle=-30.779058914538624
angle=-45.0
angle=-54.76315203118099
angle=-61.57877048694288
angle=72.133

## Generación de nube de puntos en geogebra

In [10]:
pyperclip.copy(s1.print_script())

Execute[{
"P0=(4.000,9.461)"
,"P1=(4.000,9.301)"
,"P2=(4.000,9.175)"
,"P3=(4.000,9.085)"
,"P4=(3.835,8.000)"
,"P5=(3.835,8.000)"
,"P6=(4.000,6.915)"
,"P7=(4.000,6.825)"
,"P8=(4.000,6.699)"
,"P9=(4.606,5.725)"
,"P10=(4.496,10.068)"
,"P11=(4.000,9.119)"
,"P12=(4.000,8.969)"
,"P13=(4.000,8.859)"
,"P14=(3.899,8.000)"
,"P15=(3.899,8.000)"
,"P16=(4.000,7.141)"
,"P17=(4.000,7.031)"
,"P18=(4.378,6.184)"
,"P19=(4.496,5.932)"
,"P20=(4.413,9.894)"
,"P21=(4.290,9.597)"
,"P22=(4.000,8.791)"
,"P23=(4.000,8.651)"
,"P24=(3.947,8.000)"
,"P25=(3.947,8.000)"
,"P26=(4.000,7.349)"
,"P27=(4.198,6.676)"
,"P28=(4.290,6.403)"
,"P29=(4.413,6.106)"
,"P30=(4.358,9.768)"
,"P31=(4.232,9.430)"
,"P32=(4.136,9.101)"
,"P33=(4.000,8.471)"
,"P34=(4.000,8.351)"
,"P35=(4.000,7.649)"
,"P36=(4.073,7.194)"
,"P37=(4.136,6.899)"
,"P38=(4.232,6.570)"
,"P39=(4.358,6.232)"
,"P40=(4.495,9.906)"
,"P41=(4.303,9.517)"
,"P42=(4.158,9.110)"
,"P43=(4.041,8.604)"
,"P44=(4.000,8.156)"
,"P45=(4.008,7.729)"
,"P46=(4.041,7.396)"
,"P47=(4.158,

## Checkeo de movimientos particulares

In [5]:
#Creación de objeto plataforma
p1=platform(d=6,D=10,l1=6,l2=13,max_limb_error=0.00001,max_limb_try_count=30,max_joint_angle=23*np.pi/180)
#resolución de movimiento de la plataforma
p1.solve_platform(r_x=18*np.pi/180,r_y=0*np.pi/180,h=15)
#impresión de algunos parametros
print("C1={}".format(p1.c1))
print("C2={}".format(p1.c2))
print("C3={}".format(p1.c3))

angle=180.0
roll_vector: [-1.0000000e+00  1.2246468e-16  0.0000000e+00]
C1=[ 0.         -6.14683045 15.        ]
C2=[-4.94183462  2.85316955 13.3943006 ]
C3=[ 4.94183462  2.85316955 16.6056994 ]


### Diagrama conceptual del movimiento en geogebra

In [6]:
geogebra_script=geogebra.geogebra_script()
pyperclip.copy(geogebra_script.generate_geogebra(p1))

Execute[{
"C1=(0.000,-6.147,15.000)"
,"C2=(-4.942,2.853,13.394)"
,"C3=(4.942,2.853,16.606)"
,"Polygon(C1,C2, C3)"
,"M1=(0.000,-10.000,0.000)"
,"M2=(-8.660,5.000,0.000)"
,"M3=(8.660,5.000,0.000)"
,"AUX0=(0.000,0.000,0.000)"
,"AUX4=(0.000,0.000,1.000)"
,"Plane(M1,AUX0, AUX4)"
,"Plane(M2,AUX0, AUX4)"
,"Plane(M3,AUX0, AUX4)"
,"A1=(0.000,-4.387,2.120)"
,"Segment(M1,A1)"
,"Segment(A1,C1)"
,"A2=(-3.537,2.042,1.000)"
,"Segment(M2,A2)"
,"Segment(A2,C2)"
,"A3=(4.513,2.606,3.615)"
,"Segment(M3,A3)"
,"Segment(A3,C3)"
,"AUX1=(0.000,-12.000,2.500)"
,"AUX2=(5.000,-12.000,2.500)"
,"Segment(AUX1,AUX2)"
}]
